In [1]:
import pandas as pd
import numpy as np
import scipy
import time
import matplotlib.pyplot as plt

In [37]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,TimeDistributed,Bidirectional
from keras.layers import LSTM,GRU,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D,Reshape,AveragePooling1D
from keras.optimizers import SGD,Adam

In [3]:
# Ucitavanje skupa podataka
dataset = pd.read_csv('ukupno.csv', header=0, index_col=0)

In [4]:
# uklanjamo sve kolone osim pritiska, temperature i vlaznosti za Beograd
cols = ['temperaturaBah','temperaturaBul','temperaturaMac','temperaturaMon','temperaturaCro','temperaturaHun', 'temperaturaRom', 'pritisakBah','pritisakBul','pritisakMac','pritisakMon','pritisakCro','pritisakHun', 'pritisakRom', 'vlaznostBah','vlaznostBul','vlaznostMac','vlaznostMon','vlaznostCro','vlaznostHun', 'vlaznostRom']
dataset = dataset.drop(cols,axis=1)

In [5]:
dataset.shape

(25024, 3)

In [6]:
dataset[:10]

,temperatura,vlaznost,pritisak
datumivreme,,,
1/1/18 00:00,2,0.81,1016
1/1/18 01:00,2,0.81,1016
1/1/18 01:30,2,0.81,1016
1/1/18 02:00,1,0.87,1016
1/1/18 02:30,1,0.87,1016
1/1/18 03:00,0,0.93,1015
1/1/18 03:30,-1,0.93,1015
1/1/18 04:00,-2,0.93,1015
1/1/18 04:30,-2,0.93,1015


In [7]:
values = dataset.values 
values = values.astype('float32')

In [8]:
# Prolazimo kroz podatke i pravimo 'parove' odgovrajućih ulaznih podataka sa izlaznim
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # ulazni podaci (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # predvidjanje (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    # isključujemo NaN vrednosti iz skupa podataka
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [9]:
# Normalizacija na vrednsoti u intervalu od  do 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# Uzimamo vrednosti za jedan dan, odnosno 48 redova, jer imamo podatke na pola sata
reframed = series_to_supervised(scaled, 48)

In [10]:
reframed

,var1(t-48),var2(t-48),var3(t-48),var1(t-47),var2(t-47),var3(t-47),var1(t-46),var2(t-46),var3(t-46),var1(t-45),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var1(t),var2(t),var3(t)
48,0.346939,0.773810,0.520000,0.346939,0.773810,0.520000,0.346939,0.773810,0.520000,0.326531,...,0.340000,0.428571,0.773810,0.340000,0.428571,0.773810,0.340000,0.428571,0.773810,0.320000
49,0.346939,0.773810,0.520000,0.346939,0.773810,0.520000,0.326531,0.845238,0.520000,0.326531,...,0.340000,0.428571,0.773810,0.340000,0.428571,0.773810,0.320000,0.408163,0.845238,0.320000
50,0.346939,0.773810,0.520000,0.326531,0.845238,0.520000,0.326531,0.845238,0.520000,0.306122,...,0.340000,0.428571,0.773810,0.320000,0.408163,0.845238,0.320000,0.408163,0.845238,0.300001
51,0.326531,0.845238,0.520000,0.326531,0.845238,0.520000,0.306122,0.916667,0.500000,0.285714,...,0.320000,0.408163,0.845238,0.320000,0.408163,0.845238,0.300001,0.428571,0.773810,0.300001
52,0.326531,0.845238,0.520000,0.306122,0.916667,0.500000,0.285714,0.916667,0.500000,0.265306,...,0.320000,0.408163,0.845238,0.300001,0.428571,0.773810,0.300001,0.428571,0.773810,0.300001
53,0.306122,0.916667,0.500000,0.285714,0.916667,0.500000,0.265306,0.916667,0.500000,0.265306,...,0.300001,0.428571,0.773810,0.300001,0.428571,0.773810,0.300001,0.428571,0.773810,0.300001
54,0.285714,0.916667,0.500000,0.265306,0.916667,0.500000,0.265306,0.916667,0.500000,0.265306,...,0.300001,0.428571,0.773810,0.300001,0.428571,0.773810,0.300001,0.428571,0.845238,0.280001
55,0.265306,0.916667,0.500000,0.265306,0.916667,0.500000,0.265306,1.000000,0.500000,0.285714,...,0.300001,0.428571,0.773810,0.300001,0.428571,0.845238,0.280001,0.428571,0.845238,0.280001
56,0.265306,0.916667,0.500000,0.265306,1.000000,0.500000,0.285714,0.916667,0.500000,0.285714,...,0.300001,0.428571,0.845238,0.280001,0.428571,0.845238,0.280001,0.408163,0.916667,0.260000
57,0.265306,1.000000,0.500000,0.285714,0.916667,0.500000,0.285714,0.916667,0.500000,0.285714,...,0.280001,0.428571,0.845238,0.280001,0.408163,0.916667,0.260000,0.428571,0.845238,0.260000


In [11]:
# Uklanjamo kolone za koje ne radimo predikciju
reframed.drop(reframed.columns[145:147], axis=1, inplace=True)
print(reframed.head())

    var1(t-48)  var2(t-48)  var3(t-48)  var1(t-47)  var2(t-47)  var3(t-47)  \
48    0.346939    0.773810        0.52    0.346939    0.773810        0.52   
49    0.346939    0.773810        0.52    0.346939    0.773810        0.52   
50    0.346939    0.773810        0.52    0.326531    0.845238        0.52   
51    0.326531    0.845238        0.52    0.326531    0.845238        0.52   
52    0.326531    0.845238        0.52    0.306122    0.916667        0.50   

    var1(t-46)  var2(t-46)  var3(t-46)  var1(t-45)    ...     var1(t-3)  \
48    0.346939    0.773810        0.52    0.326531    ...      0.428571   
49    0.326531    0.845238        0.52    0.326531    ...      0.428571   
50    0.326531    0.845238        0.52    0.306122    ...      0.428571   
51    0.306122    0.916667        0.50    0.285714    ...      0.428571   
52    0.285714    0.916667        0.50    0.265306    ...      0.408163   

    var2(t-3)  var3(t-3)  var1(t-2)  var2(t-2)  var3(t-2)  var1(t-1)  \
48   0.7

In [12]:
reframed.shape

(24976, 145)

In [13]:
24976-48

24928

In [14]:
dataset.index

Index(['1/1/18 00:00', '1/1/18 01:00', '1/1/18 01:30', '1/1/18 02:00',
       '1/1/18 02:30', '1/1/18 03:00', '1/1/18 03:30', '1/1/18 04:00',
       '1/1/18 04:30', '1/1/18 05:00',
       ...
       '12/21/19 07:00', '12/21/19 07:30', '12/21/19 08:00', '12/21/19 08:30',
       '12/21/19 09:00', '12/21/19 09:30', '12/21/19 10:00', '12/21/19 10:30',
       '12/21/19 11:00', '12/21/19 11:30'],
      dtype='object', name='datumivreme', length=25024)

In [15]:
dataset.index.get_loc('7/31/19 00:00')

21117

In [16]:
values = reframed.values
n_train_hours=21117-48 
train = values[:n_train_hours, :]
test = values[n_train_hours:, :] # test podaci su od 31.7.2019.
# delimo na ulazne i izlazne skupove
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(21069, 144) (21069,) (3907, 144) (3907,)


In [17]:
train_X_n = train_X.reshape((21069,48,3))
test_X_n = test_X.reshape((3907,48,3))

In [18]:
# inverzna normalizacija, kako bi mogli da uporedimo prediktovane podatke i izracunamo gresku
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, [0,2]]), axis=1)
sort_v=np.array([1,0,2])
inv_y=inv_y[:,sort_v]
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,1]

In [20]:
# Kreiranje i treniranje modela upotrebom Keras biblioteke
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(48, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1))

In [34]:
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_X_n, train_y, epochs=10, batch_size=32,validation_data=(test_X_n, test_y),shuffle=False)
yhat = model.predict(test_X_n)
# inverzna normalizacija za prediktovane vrednosti
inv_yhat = concatenate((yhat, test_X[:, [0,2,3]]), axis=1)
sort_v=np.array([1,0,2])
inv_yhat=inv_yhat[:,sort_v]
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,1]

Train on 21069 samples, validate on 3907 samples
Epoch 1/10
21069/21069 [==============================] - 3s 123us/step - loss: 1.5140 - acc: 0.0013 - val_loss: 0.4904 - val_acc: 0.0033
Epoch 2/10
21069/21069 [==============================] - 2s 118us/step - loss: 0.0864 - acc: 0.0014 - val_loss: 0.2369 - val_acc: 0.0033
Epoch 3/10
21069/21069 [==============================] - 2s 118us/step - loss: 0.0611 - acc: 0.0014 - val_loss: 0.2635 - val_acc: 0.0033
Epoch 4/10
21069/21069 [==============================] - 2s 97us/step - loss: 0.0572 - acc: 0.0014 - val_loss: 0.2047 - val_acc: 0.0033
Epoch 5/10
21069/21069 [==============================] - 2s 75us/step - loss: 0.0550 - acc: 0.0014 - val_loss: 0.2274 - val_acc: 0.0033
Epoch 6/10
21069/21069 [==============================] - 2s 84us/step - loss: 0.0475 - acc: 0.0014 - val_loss: 0.1946 - val_acc: 0.0033
Epoch 7/10
21069/21069 [==============================] - 2s 103us/step - loss: 0.0458 - acc: 0.0014 - val_loss: 0.1323 - val_

In [36]:
mae = mean_absolute_error(inv_y, inv_yhat)
print("mae =" + str(round(mae,5)))

mae =0.1285
